In [102]:
import pandas as pd
import numpy as np

In [103]:
import warnings
warnings.filterwarnings('ignore')

In [104]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [105]:
df_hindi = pd.read_csv('hindi movies/indian_movies.csv')

In [106]:
df_hindi

,ID,Movie Name,Year,Timing(min),Rating(10),Votes,Genre,Language
0,tt0398974,Dr. Shaitan,1960,-,-,-,-,hindi
1,tt1702558,Nadir Khan,1968,-,-,-,-,urdu
2,tt0493437,Apna Sapna Money Money,2006,134 min,5.3,"1,892","Comedy, Musical, Romance",hindi
3,tt0273405,Aag Aur Sholay,1987,-,2.2,20,-,urdu
4,tt0049595,Parivar,1956,-,7.4,21,"Comedy, Drama, Family",hindi
...,...,...,...,...,...,...,...,...
50597,tt2784720,Sulagte Armaan,1984,-,-,-,Drama,hindi
50598,tt0315201,Aranya,1971,107 min,-,-,Drama,assamese
50599,tt0270221,Balayogini,1936,120 min,-,-,-,telugu
50600,tt0388697,Alajadi,1990,133 min,-,-,Drama,telugu


# DATA PREPROCESSING

drop duplicates

In [107]:
df_movies = df_hindi.drop_duplicates()
df_movies.duplicated().sum()

0

### clear year column.

In [108]:
# Step 1: Extract the 4-digit year using regex
df_movies['Year'] = df_movies['Year'].str.extract('(\d{4})')[0]

# Step 2: Convert the 'Year' column to numeric with error handling
df_movies['Year'] = pd.to_numeric(df_movies['Year'], errors='coerce')



In [109]:
print(df_movies['Year'].unique())

[1960. 1968. 2006. 1987. 1956. 2008. 2019. 2007. 1939. 1992. 1997. 2018.
 2016. 1976. 1958. 1986. 2013. 2017. 1988. 2012. 1974. 1991. 1990. 2020.
 2009. 1978. 1999. 1935. 1970. 1985. 1981. 2000.   nan 2003. 1967. 1933.
 1996. 2010. 1995. 1973. 2015. 1950. 1966. 2021. 1994. 1989. 1938. 1969.
 1984. 1952. 1977. 1957. 2014. 1937. 2005. 1982. 1983. 1965. 2004. 1959.
 2011. 1961. 1971. 1962. 2002. 1940. 2022. 1979. 1942. 2001. 1947. 1953.
 1998. 1963. 1954. 1980. 1975. 1964. 1948. 1943. 1932. 1949. 1951. 1972.
 1934. 1941. 1955. 1946. 1993. 1944. 1936. 1945. 1931. 1920. 1919. 2024.
 1917. 1929. 1915. 1918. 1930. 1914. 2023. 1913.]


In [110]:
# Step 1: Drop any rows where 'Year' is NaN
df_movies = df_movies.dropna(subset=['Year'])

# Step 2: Convert the 'Year' column from float to integer type
df_movies['Year'] = df_movies['Year'].astype(int)

# Verify the changes
print(df_movies['Year'].head())
print(df_movies['Year'].dtype)  # Check the data type to ensure it's integer


0    1960
1    1968
2    2006
3    1987
4    1956
Name: Year, dtype: int32
int32


### Clear Timing(min) column

In [111]:
# Remove 'min', commas, and convert to float; handle invalid entries as NaN
df_movies['Timing(min)'] = (
    df_movies['Timing(min)']
    .str.replace(',', '', regex=False)
    .str.replace(' min', '', regex=False)
    .replace('-', np.nan)
    .astype(float)
)

### Clear Rating column

In [112]:
#Convert 'Rating(10)' to numeric, invalid values set to NaN
df_movies['Rating(10)'] = pd.to_numeric(df_movies['Rating(10)'], errors='coerce')

### Clear votes column

In [113]:
import re

# Function to clean and convert 'Votes' to numeric
def clean_votes(vote):
    if isinstance(vote, str):
        # Remove commas and dollar signs
        vote = re.sub(r'[,$]', '', vote)
        # Handle cases like '5.16M' (convert to millions)
        if 'M' in vote:
            return float(vote.replace('M', '')) * 1_000_000
        elif 'K' in vote:
            return float(vote.replace('K', '')) * 1_000
    # Convert to float or return NaN for invalid cases
    try:
        return float(vote)
    except ValueError:
        return np.nan

# Apply the cleaning function to the 'Votes' column
df_movies['Votes'] = df_movies['Votes'].apply(clean_votes)

# Verify the result
print(df_movies['Votes'].head())


0       NaN
1       NaN
2    1892.0
3      20.0
4      21.0
Name: Votes, dtype: float64


### clear genre column

In [114]:
#handle multiple values entry in the column
df_movies['main_genres'] = df_movies['Genre'].str.split(',').str[0]

#3. Replace any empty values in the 'main_genre' column with NaN (Not a Number)
df_movies['main_genres'] = df_movies['main_genres'].replace('', np.nan)
df_movies['main_genres']

0                        -
1                        -
2                   Comedy
3                        -
4                   Comedy
               ...        
50597    Drama            
50598    Drama            
50599                    -
50600    Drama            
50601    Drama            
Name: main_genres, Length: 48909, dtype: object

Handle missing values

In [115]:
df_movies = df_movies.dropna(subset=['Year', 'Timing(min)', 'Rating(10)', 'Votes'])

In [116]:
df_movies

,ID,Movie Name,Year,Timing(min),Rating(10),Votes,Genre,Language,main_genres
2,tt0493437,Apna Sapna Money Money,2006,134.0,5.3,1892.0,"Comedy, Musical, Romance",hindi,Comedy
6,tt6597160,Jacqueline I Am Coming,2019,112.0,7.9,16.0,Drama,hindi,Drama
7,tt0829459,A Mighty Heart,2007,108.0,6.6,26885.0,"Biography, Drama, History",urdu,Biography
9,tt0364628,Raktalekha,1992,175.0,6.3,12.0,Drama,bengali,Drama
10,tt0364733,Veedevadandi Babu,1997,138.0,5.6,218.0,Comedy,telugu,Comedy
...,...,...,...,...,...,...,...,...,...
50586,tt9277948,Akshara,2021,136.0,6.5,90.0,Thriller,telugu,Thriller
50591,tt0248201,Mudhal Thedi,1955,145.0,6.8,12.0,-,tamil,-
50592,tt6836936,Saaho,2019,170.0,5.2,16102.0,"Action, Thriller",telugu,Action
50594,tt0073191,Jai Santoshi Maa,1975,145.0,6.3,114.0,"Drama, Fantasy",hindi,Drama


In [117]:
df_movies.drop(['ID', 'Genre'], axis=1, inplace=True)

set movis name column as dataframe since to apply minmaxscaler function all columns need to be in numerical format

In [118]:
# Set the 'title' column as the DataFrame index
df_movies.set_index('Movie Name', inplace=True)

In [119]:
df_movies

,Year,Timing(min),Rating(10),Votes,Language,main_genres
Movie Name,,,,,,
Apna Sapna Money Money,2006,134.0,5.3,1892.0,hindi,Comedy
Jacqueline I Am Coming,2019,112.0,7.9,16.0,hindi,Drama
A Mighty Heart,2007,108.0,6.6,26885.0,urdu,Biography
Raktalekha,1992,175.0,6.3,12.0,bengali,Drama
Veedevadandi Babu,1997,138.0,5.6,218.0,telugu,Comedy
...,...,...,...,...,...,...
Akshara,2021,136.0,6.5,90.0,telugu,Thriller
Mudhal Thedi,1955,145.0,6.8,12.0,tamil,-
Saaho,2019,170.0,5.2,16102.0,telugu,Action


### Encode categorical features

In [120]:
#1. Create dummy variables for categorical columns 
dummies = pd.get_dummies(df_movies[['Language', 'main_genres']], drop_first=True)
dummies.astype(int)


,Language_bengali,Language_bhojpuri,Language_gujarati,Language_hindi,Language_kannada,Language_kashmiri,Language_konkani,Language_malayalam,Language_marathi,Language_nepali,...,main_genres_Romance,main_genres_Sci-Fi,main_genres_Sci-Fi,main_genres_Short,main_genres_Short,main_genres_Sport,main_genres_Talk-Show,main_genres_Thriller,main_genres_Thriller,main_genres_War
Movie Name,,,,,,,,,,,,,,,,,,,,,
Apna Sapna Money Money,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jacqueline I Am Coming,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Mighty Heart,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Raktalekha,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Veedevadandi Babu,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Akshara,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Mudhal Thedi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Saaho,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
#2. Concatenate the dummy variables with the original DataFrame i.e. concat the above columns with df_movies columns. 
df_movies_dum = pd.concat([df_movies, dummies], axis=1)

In [122]:
df_movies_dum.drop(['Language', 'main_genres'], axis=1, inplace=True)

In [123]:
df_movies_dum

,Year,Timing(min),Rating(10),Votes,Language_bengali,Language_bhojpuri,Language_gujarati,Language_hindi,Language_kannada,Language_kashmiri,...,main_genres_Romance,main_genres_Sci-Fi,main_genres_Sci-Fi,main_genres_Short,main_genres_Short,main_genres_Sport,main_genres_Talk-Show,main_genres_Thriller,main_genres_Thriller,main_genres_War
Movie Name,,,,,,,,,,,,,,,,,,,,,
Apna Sapna Money Money,2006,134.0,5.3,1892.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
Jacqueline I Am Coming,2019,112.0,7.9,16.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
A Mighty Heart,2007,108.0,6.6,26885.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Raktalekha,1992,175.0,6.3,12.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Veedevadandi Babu,1997,138.0,5.6,218.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Akshara,2021,136.0,6.5,90.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
Mudhal Thedi,1955,145.0,6.8,12.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Saaho,2019,170.0,5.2,16102.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [124]:
df_movies_dum

,Year,Timing(min),Rating(10),Votes,Language_bengali,Language_bhojpuri,Language_gujarati,Language_hindi,Language_kannada,Language_kashmiri,...,main_genres_Romance,main_genres_Sci-Fi,main_genres_Sci-Fi,main_genres_Short,main_genres_Short,main_genres_Sport,main_genres_Talk-Show,main_genres_Thriller,main_genres_Thriller,main_genres_War
Movie Name,,,,,,,,,,,,,,,,,,,,,
Apna Sapna Money Money,2006,134.0,5.3,1892.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
Jacqueline I Am Coming,2019,112.0,7.9,16.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
A Mighty Heart,2007,108.0,6.6,26885.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Raktalekha,1992,175.0,6.3,12.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Veedevadandi Babu,1997,138.0,5.6,218.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Akshara,2021,136.0,6.5,90.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
Mudhal Thedi,1955,145.0,6.8,12.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Saaho,2019,170.0,5.2,16102.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Scaling (minmaxscaler)

In [125]:
# Apply MinMaxScaler to scale the data for model training
scaler = MinMaxScaler() #create object for minmaxscalar library

df_scaled = scaler.fit_transform(df_movies_dum) #new data created

#data created is in numpy array form so convert it back to dataframe form
df_scaled = pd.DataFrame(df_scaled, columns=df_movies_dum.columns)

# Display the scaled DataFrame

df_scaled


,Year,Timing(min),Rating(10),Votes,Language_bengali,Language_bhojpuri,Language_gujarati,Language_hindi,Language_kannada,Language_kashmiri,...,main_genres_Romance,main_genres_Sci-Fi,main_genres_Sci-Fi,main_genres_Short,main_genres_Short,main_genres_Sport,main_genres_Talk-Show,main_genres_Thriller,main_genres_Thriller,main_genres_War
0,0.852941,0.057743,0.477778,0.001976,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.980392,0.048119,0.766667,0.000012,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.862745,0.046369,0.622222,0.028151,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.715686,0.075678,0.588889,0.000007,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.764706,0.059493,0.511111,0.000223,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16892,1.000000,0.058618,0.611111,0.000089,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16893,0.352941,0.062555,0.644444,0.000007,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16894,0.980392,0.073491,0.466667,0.016858,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16895,0.549020,0.062555,0.588889,0.000114,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# DBSCAN algorithm

In [126]:
eps_array = [0.2, 0.5, 1]  # List of different epsilon values 
min_samples_array = [5, 10, 30]  # List of different min_samples values 

In [127]:

# Iterate over each combination of eps and min_samples
for eps in eps_array:
    for min_samples in min_samples_array:
        # Initialize and fit the DBSCAN model with the current parameters
        #we pass the eps and min_samples value from loop as parameter into this function
        clusterer = DBSCAN(eps=eps, min_samples=min_samples).fit(df_scaled)
        
        # Retrieve all the cluster using 'labels_' parameter from the fitted model
        cluster_labels = clusterer.labels_
        
        # Check if the algorithm found only one cluster or marked all points as noise (-1 label for noise)
        if len(set(cluster_labels)) == 1:
            continue  # Skip this combination as it does not provide meaningful clusters
        
        #if more than on cluster is generated i.e. no noise proper cluster formed then calculate silhoutte score
        # Calculate the silhouette score to evaluate the quality of the clustering
        silhouette_avg = silhouette_score(df_scaled, cluster_labels)
        
        # Print the current parameters, number of clusters, and the silhouette score
        print("For eps =", eps,
              "For min_samples =", min_samples,
              "Count clusters =", len(set(cluster_labels)), #to get unique clusters only use set and to calculate the no of clusters use len function
              "The average silhouette_score is :", silhouette_avg)


For eps = 0.2 For min_samples = 5 Count clusters = 197 The average silhouette_score is : 0.704814026843598
For eps = 0.2 For min_samples = 10 Count clusters = 146 The average silhouette_score is : 0.6876065037016953
For eps = 0.2 For min_samples = 30 Count clusters = 83 The average silhouette_score is : 0.6080803953103275
For eps = 0.5 For min_samples = 5 Count clusters = 214 The average silhouette_score is : 0.733584390879883
For eps = 0.5 For min_samples = 10 Count clusters = 167 The average silhouette_score is : 0.7190604148723744
For eps = 0.5 For min_samples = 30 Count clusters = 92 The average silhouette_score is : 0.656948452623421
For eps = 1 For min_samples = 5 Count clusters = 210 The average silhouette_score is : 0.671204522368312
For eps = 1 For min_samples = 10 Count clusters = 164 The average silhouette_score is : 0.6590465050207542
For eps = 1 For min_samples = 30 Count clusters = 91 The average silhouette_score is : 0.629026786443655


maximum silhoutte score comes where eps = 0.5 and min_sample = 5

In [128]:
dbscan_model = DBSCAN(eps=0.5, min_samples=5).fit(df_scaled)
print("For eps =", 0.5,
      "For min_samples =", 5,
      "Count clusters =", len(set(dbscan_model.labels_)),
      "The average silhouette_score is :", silhouette_score(df_scaled, dbscan_model.labels_))

For eps = 0.5 For min_samples = 5 Count clusters = 214 The average silhouette_score is : 0.733584390879883


In [129]:
#create a new column in df_movies as dbscan_clusters which will store the cluster to which the particular movie belong to. -1 means the movie belong to noise point (considered as outlier)
df_movies['dbscan_clusters'] = dbscan_model.labels_
df_movies

,Year,Timing(min),Rating(10),Votes,Language,main_genres,dbscan_clusters
Movie Name,,,,,,,
Apna Sapna Money Money,2006,134.0,5.3,1892.0,hindi,Comedy,0
Jacqueline I Am Coming,2019,112.0,7.9,16.0,hindi,Drama,1
A Mighty Heart,2007,108.0,6.6,26885.0,urdu,Biography,2
Raktalekha,1992,175.0,6.3,12.0,bengali,Drama,3
Veedevadandi Babu,1997,138.0,5.6,218.0,telugu,Comedy,4
...,...,...,...,...,...,...,...
Akshara,2021,136.0,6.5,90.0,telugu,Thriller,75
Mudhal Thedi,1955,145.0,6.8,12.0,tamil,-,30
Saaho,2019,170.0,5.2,16102.0,telugu,Action,10


In [130]:
min_value = df_movies['Year'].min()
max_value = df_movies['Year'].max()

print(f'Minimum value in column: {min_value}')
print(f'Maximum value in column: {max_value}')

Minimum value in column: 1919
Maximum value in column: 2021


# Movie recommendation function

In [131]:
import random

#fucntion will take movie name as argument
def recommend_movie(movie_name: str):
    # Convert the input movie name to lowercase for case-insensitive matching
    movie_name = movie_name.lower()

    # Create a new column 'name' with lowercase movie names for comparison
    df_movies['name'] = df_movies.index.str.lower()

    # Find the movie that matches the input name
    movie = df_movies[df_movies['name'].str.contains(movie_name, na=False)]

    if not movie.empty:
        # Get the cluster label of the input movie. i.e. match the cluster value of input movie and the movies that match the input movies.
        cluster = movie['dbscan_clusters'].values[0]

        # Get all movies in the same cluster
        cluster_movies = df_movies[df_movies['dbscan_clusters'] == cluster]

        # If there are more than 5 movies in the cluster, randomly select 5
        #if len(cluster_movies) >= 5:
            #recommended_movies = random.sample(list(cluster_movies.index), 5)
        #else:
            # If fewer than 5, return all the movies in the cluster
        recommended_movies = list(cluster_movies.index)

        # Print the recommended movies
        print('--- We can recommend you these movies ---')
        for m in recommended_movies:
            print(m)
    else:
        print('Movie not found in the database.')

In [132]:
s = input("Enter movie name")
print("\n\n")
recommend_movie(s)




--- We can recommend you these movies ---
Maithaanam
Theeviram
Thiranthidu Seese
Oru Naal Iravil
Priyasakhi
Endrendrum Kadhal
Poomagal Oorvalam
Neeya
V1 Murder Case
Thanga Padhakkam
Unnal Mudiyum Thambi
Kattradhu Thamizh
Kalam
Ninaithen Vandhai
Maayanadhi
Meipporul
Vinnaithaandi Varuvaayaa
Manaivi Ready
Neethaane En Ponvasantham
Adithya Varma
Kutty Pisasu
Jeeva
Manithan
Iyarkai
Idhuvum Kadandhu Pogum
Ammani
Bhuvana Oru Kelvikkuri
Kadhalukku Mariyadhai
Ennul Aayiram
Taramani
Kasthuriman
Mithivedi
Anbirkiniyal
Abhiyude Katha Anuvinteyum
Eppadi Manasukkul Vandhai
Pathinaru Vayathinile
Chinna Thambi
Avargal
Vazhkai
Kaadu
Youth
Madurai Sambavam
Dowry Kalyanam
Julie Ganapathy
Paradesi
Naalaiya Theerpu
Azhakan
Kargil
Oorellam Un Paattu
Oru Nadigayin Vakku Moolam
Rummy
Kadhal Mannan
Pisasu
Avvaiyyar
Love Today
Nallavanukku Nallavan
Kalathil Sandhippom
Kadhalar Dinam
Vaidehi Kaathirundaal
Thillana Mohanambal
Phoonk
Kallappadam
Kanni Maadam
Sommokadidhi Sokokadidhi
Karu
Swathi Muthyam
Kannum 